In [12]:
import pandas as pd
import uuid
import string
import os

## script parameters.

In [8]:
table_name = "Human Resources Setup"
table_id = "5218"
xml_port_id = "51000"
exported_table_file_path = "report_reader/Human Resource Setup.csv"
output_file_path = "report_reader/outer.txt"


## Functions definations

In [9]:
def clean_str(text):
    punctuation = string.punctuation + " "
    punctuation = punctuation.replace(".", "")
    for char in punctuation:
        if char in punctuation:
            text = text.replace(char, "")
    return text

def make_elem(row):
    field_name, data_type = row[["Field Name","Data Type"]]
    uuid_code = str(uuid.uuid4()).upper()
    elem = "    { [{"+uuid_code+"}];2 ;"+clean_str(field_name)+";Element ;Field   ;DataType="+data_type+"; SourceField="+table_name+"::"+field_name+" }"
    return elem

def gen_elements(df):
    return "\n".join(df.apply(make_elem, axis=1).to_list())

## Generate xmlport and save it out. 

In [13]:
df = pd.read_csv(exported_table_file_path)
elem = gen_elements(df)

content = """
OBJECT XMLport """+xml_port_id+""" Import HRSetup
{
  OBJECT-PROPERTIES
  {
    Date=25/10/16;
    Time=12:00:00;
    Version List=NAVW110.0;
  }
  PROPERTIES
  {
    CaptionML=ENU=Import Payroll;
    Direction=Import;
    Format/Evaluate=C/SIDE Format/Evaluate;
    TextEncoding=WINDOWS;
    OnInitXMLport=BEGIN
                    I := 0;
                  END;

    Format=Variable Text;
    FieldDelimiter=<None>;
    FieldSeparator=<TAB>;
  }
  ELEMENTS
  {
    { [{C39C2F1A-F410-4CD3-B5A2-EA812345D9CB}];  ;Root                ;Element ;Text     }

    { [{04E61061-50EA-4A5C-B805-9F8582579ADB}];1 ;"""+clean_str(table_name)+"""       ;Element ;Table   ;
                                                  SourceTable=Table"""+table_id+""";
                                                  Import::OnBeforeInsertRecord=BEGIN
                                                                                 
                                                                               END;
                                                                                }
""" + elem + """
 }
  EVENTS
  {
  }
  REQUESTPAGE
  {
    PROPERTIES
    {
    }
    CONTROLS
    {
    }
  }
  CODE
  {
    VAR
      I@1000 : Integer;

    BEGIN
    END.
  }
}
"""

with open(output_file_path, "w") as f:
    f.write(content)
    
print(f"Created and saved at '{output_file_path}'!!")
os.system("notepad "+output_file_path)

Created and saved at 'report_reader/outer.txt'!!


0